# Exercise 1: Linear Regression

### This notebook is executed automatically. Failing to meet any of the submission requirements will result in a 25 point fine or your submission not being graded at all. Kindly reminder: the homework assignments grade is 50% of the final grade.

### Make sure you restart the notebook and check the filename before submission. Appeals based on wrong filenames and errors due to syntax and execution errors will not be accepted.

### Do not start the exercise until you fully understand the submission guidelines.

## Read the following instructions carefully:

1. This jupyter notebook contains all the step-by-step instructions needed for this exercise.
1. Submission includes this notebook only with the exercise number and your ID as the filename. For example: `hw1_123456789_987654321.ipynb` if you submitted in pairs and `hw1_123456789.ipynb` if you submitted the exercise alone.
1. Write **efficient vectorized** code whenever possible. Some calculations in this exercise take several minutes when implemented efficiently, and might take much longer otherwise. Unnecessary loops will result in point deduction.
1. You are responsible for the correctness of your code and should add as many tests as you see fit. Tests will not be graded nor checked.
1. Write your functions in this notebook only. **Do not create Python modules and import them**.
1. You are allowed to use functions and methods from the [Python Standard Library](https://docs.python.org/3/library/) and [numpy](https://www.numpy.org/devdocs/reference/) only. **Do not import anything else.**
1. Your code must run without errors. Make sure your `numpy` version is at least 1.15.4 and that you are using at least python 3.6. Changes of the configuration we provided are at your own risk. Any code that cannot run will not be graded.
1. Write your own code. Cheating will not be tolerated.
1. Answers to qualitative questions should be written in **markdown** cells (with $\LaTeX$ support). Answers that will be written in commented code blocks will not be checked.

## In this exercise you will perform the following:
1. Load a dataset and perform basic data exploration using a powerful data science library called [pandas](https://pandas.pydata.org/pandas-docs/stable/).
1. Preprocess the data for linear regression.
1. Compute the cost and perform gradient descent in pure numpy in vectorized form.
1. Fit a linear regression model using a single feature.
1. Visualize your results using matplotlib.
1. Perform multivariate linear regression.
1. Perform polynomial regression.
1. Experiment with adaptive learning rates.

# I have read and understood the instructions: ***203459011*** ***318633294***

In [ ]:
import numpy as np # used for scientific computing
import pandas as pd # used for data analysis and manipulation
import matplotlib.pyplot as plt # used for visualization and plotting

np.random.seed(42) 

# make matplotlib figures appear inline in the notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = (14.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

## Part 1: Data Preprocessing (10 Points)

For the following exercise, we will use a dataset containing housing prices in King County, USA. The dataset contains 5,000 observations with 18 features and a single target value - the house price. 

First, we will read and explore the data using pandas and the `.read_csv` method. Pandas is an open source library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.

In [ ]:
# Read comma separated data
df = pd.read_csv('data.csv') # Make sure this cell runs regardless of your absolute path.
# df stands for dataframe, which is the default format for datasets in pandas

### Data Exploration
A good practice in any data-oriented project is to first try and understand the data. Fortunately, pandas is built for that purpose. Start by looking at the top of the dataset using the `df.head()` command. This will be the first indication that you read your data properly, and that the headers are correct. Next, you can use `df.describe()` to show statistics on the data and check for trends and irregularities.

In [ ]:
df.head(6)

In [ ]:
df.describe()

We will start with one variable linear regression by extracting the target column and the `sqft_living` variable from the dataset. We use pandas and select both columns as separate variables and transform them into a numpy array.

In [ ]:
X = df['sqft_living'].values
y = df['price'].values

## Preprocessing

As the number of features grows, calculating gradients gets computationally expensive.
We can speed this up by normalizing the input data to ensure all values are within the same range.
This is especially important for datasets with high standard deviations or differences in the ranges of the attributes.
Use [mean normalization](https://en.wikipedia.org/wiki/Feature_scaling) for the features (`X`) and the true labels (`y`).

Implement the cost function `preprocess`.

In [ ]:
def preprocess(X, y):
    """
    Perform mean normalization on the features and true labels.

    Input:
    - X: Inputs (n features over m instances).
    - y: True labels.

    Returns two vales:
    - X: The mean normalized inputs.
    - y: The mean normalized labels.
    """
    ###########################################################################
    # TODO: Implement the normalization function.                             #
    ###########################################################################

    X = (X - np.mean(X, axis = 0)) / (np.amax(X, axis = 0) - np.amin(X, axis = 0))
    y = (y - y.mean()) / (y.max() - y.min())
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return X, y

In [ ]:
X, y = preprocess(X, y)

We will split the data into two datasets: 
1. The training dataset will contain 80% of the data and will always be used for model training.
2. The validation dataset will contain the remaining 20% of the data and will be used for model evaluation. For example, we will pick the best alpha and the best features using the validation dataset, while still training the model using the training dataset.

In [ ]:
# training and validation split
np.random.seed(42)
indices = np.random.permutation(X.shape[0])
idx_train, idx_val = indices[:int(0.8*X.shape[0])], indices[int(0.8*X.shape[0]):]
X_train, X_val = X[idx_train], X[idx_val]
y_train, y_val = y[idx_train], y[idx_val]

## Data Visualization
Another useful tool is data visualization. Since this problem has only two parameters, it is possible to create a two-dimensional scatter plot to visualize the data. Note that many real-world datasets are highly dimensional and cannot be visualized naively. We will be using `matplotlib` for all data visualization purposes since it offers a wide range of visualization tools and is easy to use.

In [ ]:
plt.plot(X_train, y_train, 'ro', ms=1, mec='k') # the parameters control the size, shape and color of the scatter plot
plt.ylabel('Price in USD')
plt.xlabel('sq.ft')
plt.show()

## Bias Trick

Make sure that `X` takes into consideration the bias $\theta_0$ in the linear model. Hint, recall that the predications of our linear model are of the form:

$$
\hat{y} = h_\theta(x) = \theta^T x = \theta_0 + \theta_1 x_1
$$

Add columns of ones as the zeroth column of the features (do this for both the training and validation sets).

In [ ]:
###########################################################################
#                            START OF YOUR CODE                           #
###########################################################################

X_train = np.column_stack((np.ones(X_train.shape[0]), X_train))
X_val = np.column_stack((np.ones(X_val.shape[0]), X_val))

###########################################################################
#                             END OF YOUR CODE                            #
###########################################################################

## Part 2: Single Variable Linear Regression (40 Points)
Simple linear regression is a linear regression model with a single explanatory variable and a single target value.

$$
\hat{y} = h_\theta(x) = \theta^T x = \theta_0 + \theta_1 x_1
$$

## Gradient Descent 

Our task is to find the best possible linear line that explains all the points in our dataset. We start by guessing initial values for the linear regression parameters $\theta$ and updating the values using gradient descent. 

The objective of linear regression is to minimize the cost function $J$:

$$
J(\theta) = \frac{1}{2m} \sum_{i=1}^{n}(h_\theta(x^{(i)})-y^{(i)})^2
$$

where the hypothesis (model) $h_\theta(x)$ is given by a **linear** model:

$$
h_\theta(x) = \theta^T x = \theta_0 + \theta_1 x_1
$$

$\theta_j$ are parameters of your model. and by changing those values accordingly you will be able to lower the cost function $J(\theta)$. One way to accomplish this is to use gradient descent:

$$
\theta_j = \theta_j - \alpha \frac{1}{m} \sum_{i=1}^m (h_\theta(x^{(i)})-y^{(i)})x_j^{(i)}
$$

In linear regression, we know that with each step of gradient descent, the parameters $\theta_j$ get closer to the optimal values that will achieve the lowest cost $J(\theta)$.

Implement the cost function `compute_cost`. (10 points)

In [ ]:
t = [2 , 1]

In [ ]:
def compute_cost(X, y, theta):
    """
    Computes the average squared difference between an observation's actual and
    predicted values for linear regression.  

    Input:
    - X: inputs  (n features over m instances).
    - y: true labels (1 value over m instances).
    - theta: the parameters (weights) of the model being learned.

    Returns a single value:
    - J: the cost associated with the current set of parameters (single number).
    """
    
    J = 0  # Use J for the cost.
    ###########################################################################
    # Implement the MSE cost function.                                  #
    ###########################################################################
    # compute dot mult as vectors product of theta (as a vector) and X and reduce y (by coordinate)
    dist = X.dot(theta) - y
    dist = np.square(dist)
    J = np.sum(dist) / (2 * X.shape[0])

    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return J

In [ ]:
theta = np.array([-1, 2])
J = compute_cost(X_train, y_train, theta)

Implement the gradient descent function `gradient_descent`. (10 points)

In [ ]:
def gradient_descent(X, y, theta, alpha, num_iters):
    """
    Learn the parameters of the model using gradient descent using 
    the *training set*. Gradient descent is an optimization algorithm 
    used to minimize some (loss) function by iteratively moving in 
    the direction of the steepest descent as defined by the negative of
    the gradient. We use gradient descent to update the parameters
    (weights) of our model.

    Input:
    - X: Inputs  (n features over m instances).
    - y: True labels (1 value over m instances).
    - theta: The parameters (weights) of the model being learned.
    - alpha: The learning rate of your model.
    - num_iters: The number of updates performed.

    Returns two values:
    - theta: The learned parameters of your model.
    - J_history: the loss value for every iteration.
    """
    
    J_history = [] # Use a python list to save cost in every iteration
    theta = theta.copy() # avoid changing the original thetas
    ###########################################################################
    #                     Gradient descent optimization algorithm.            #
    ###########################################################################
    gradient =  0
    
    for i in range(num_iters):
        gradient = X.dot(theta) - y
        # new alpha is changing iff decay is given to function
        theta = theta - alpha * (1/X.shape[0]) * X.T.dot(gradient)
        J_history.insert(i,compute_cost(X, y, theta))

    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return theta, J_history

In [ ]:
np.random.seed(42)
theta = np.random.random(size=2)
iterations = 40000
alpha = 0.1
theta, J_history = gradient_descent(X_train ,y_train, theta, alpha, iterations)

You can evaluate the learning process by monitoring the loss as training progress. In the following graph, we visualize the loss as a function of the iterations. This is possible since we are saving the loss value at every iteration in the `J_history` array. This visualization might help you find problems with your code. Notice that since the network converges quickly, we are using logarithmic scale for the number of iterations. 

In [ ]:
plt.plot(np.arange(iterations), J_history)
plt.xscale('log')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.title('Loss as a function of iterations')
plt.show()

Implement the pseudo-inverse function `pinv`. **Do not use `np.linalg.pinv`**, instead use only direct matrix multiplication as you saw in class (you can calculate the inverse of a matrix using `np.linalg.inv`). (10 points)

In [ ]:
def pinv(X, y):
    """
    Calculate the optimal values of the parameters using the pseudo-inverse
    approach as you saw in class using the *training set*.

    Input:
    - X: Inputs  (n features over m instances).
    - y: True labels (1 value over m instances).

    Returns two values:
    - theta: The optimal parameters of your model.

    ########## DO NOT USE np.linalg.pinv ##############
    """
    
    pinv_theta = []
    ###########################################################################
    #  Implement the pseudo-inverse algorithm.                            #
    ###########################################################################
    pinv_theta = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return pinv_theta

In [ ]:
theta_pinv = pinv(X_train ,y_train)
J_pinv = compute_cost(X_train, y_train, theta_pinv)


We can add the loss value for the theta calculated using the pseudo-inverse to our graph. This is another sanity check as the loss of our model should converge to the pseudo-inverse loss.

In [ ]:
plt.plot(np.arange(iterations), J_history)
plt.xscale('log')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.title('Loss as a function of iterations')
plt.hlines(y = J_pinv, xmin = 0, xmax = len(J_history), color='r',
           linewidth = 1, linestyle = 'dashed')
plt.show()

We can use a better approach for the implementation of `gradient_descent`. Instead of performing 40,000 iterations, we wish to stop when the improvement of the loss value is smaller than `1e-8` from one iteration to the next. Implement the function `efficient_gradient_descent`. (5 points)

In [ ]:
def efficient_gradient_descent(X, y, theta, alpha, num_iters):
    """
    Learn the parameters of your model using the *training set*, but stop 
    the learning process once the improvement of the loss value is smaller 
    than 1e-8. This function is very similar to the gradient descent 
    function you already implemented.

    Input:
    - X: Inputs  (n features over m instances).
    - y: True labels (1 value over m instances).
    - theta: The parameters (weights) of the model being learned.
    - alpha: The learning rate of your model.
    - num_iters: The number of updates performed.

    Returns two values:
    - theta: The learned parameters of your model.
    - J_history: the loss value for every iteration.
    """
    
    J_history = [] # Use a python list to save cost in every iteration
    theta = theta.copy() # avoid changing the original thetas
    MIN_DIFF = 1e-8
    ###########################################################################
    #                 The gradient descent optimization algorithm.            #
    ###########################################################################
    for i in range(num_iters):
        #check previous cost to current one, break loop if less than MIN_DIFF or cur_cost > prev cost
        #same code from gradient_descent() above
        gradient = X.dot(theta) - y
        # new alpha is changing iff decay is given to function
        theta = theta - alpha * (1/X.shape[0]) * X.T.dot(gradient)
        J_history.append(compute_cost(X, y, theta))
        if (i > 0) and (J_history[-2] - J_history[-1] < MIN_DIFF):
            break

    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return theta, J_history

In [ ]:
#-----------------------------
# TEST
np.random.seed(42)
theta = np.random.random(size=2)

iterations = 40000
alpha = 0.1
theta, J_history_eff = efficient_gradient_descent(X_train ,y_train, theta, alpha, iterations)
print(J_history_eff)
print(len(J_history_eff))



The learning rate is another factor that determines the performance of our model in terms of speed and accuracy. Complete the function `find_best_alpha`. Make sure you use the training dataset to learn the parameters (thetas) and use those parameters with the validation dataset to compute the cost.

In [ ]:
def find_best_alpha(X_train, y_train, X_val, y_val, iterations):
    """
    Iterate over provided values of alpha and train a model using the 
    *training* dataset. maintain a python dictionary with alpha as the 
    key and the loss on the *validation* set as the value.

    Input:
    - X_train, y_train, X_val, y_val: the training and validation data
    - iterations: maximum number of iterations

    Returns:
    - alpha_dict: A python dictionary - {key (alpha) : value (validation loss)}
    """
    
    alphas = [0.00001, 0.00003, 0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 2, 3]
    alpha_d = dict()
    np.random.seed(42)
    theta = np.random.random(size=X_train.shape[1])
    for alpha in alphas:
#        print("before training theta =" + theta) #test
        theta_cur, _ = efficient_gradient_descent(X_train ,y_train, theta, alpha, iterations)
        #print("After training theta =" + theta_cur) #test
        alpha_d[alpha] = compute_cost(X_val, y_val, theta_cur)
        # print(f"alpha={alpha}, cost={alpha_d[alpha]}") #test
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return alpha_d

In [ ]:
alpha_dict = find_best_alpha(X_train, y_train, X_val, y_val, 40000)

Obtain the best learning rate from the dictionary `alpha_dict`. This can be done in a single line using built-in functions.

In [ ]:
best_alpha = None
###########################################################################
#                            START OF YOUR CODE                           #
###########################################################################
best_alpha = min(alpha_dict, key=alpha_dict.get)


###########################################################################
#                             END OF YOUR CODE                            #
###########################################################################


Pick the best three alpha values you just calculated and provide **one** graph with three lines indicating the training loss as a function of iterations (Use 10,000 iterations). Note you are required to provide general code for this purpose (no hard-coding). Make sure the visualization is clear and informative. (5 points)

In [ ]:
###########################################################################
#                            START OF YOUR CODE                           #
###########################################################################
sorted_d = sorted(alpha_dict, key=alpha_dict.get)
three_smallest_alphas = [min1, min2, min3] = sorted_d[:3]
theta_arr = []
J_history_array = []
# start from the same theta by seeded random
np.random.seed(42)
theta_1 = np.random.random(size=2)
iterations = 10_000
# training for each alpha
for alpha in three_smallest_alphas:
    theta_temp, J_eff = efficient_gradient_descent(X_train ,y_train, theta_1, alpha, iterations)
    J_history_array.append(J_eff)
    theta_arr.append(theta_temp)

#store the trained theta from the best alpha for sanity check ahead
theta = theta_arr[0]

#plot 3 histograms
for i in range(3): # [1-3]
    plt.plot(np.arange(len(J_history_array[i])), J_history_array[i], label=f'min{i+1}-alpha={three_smallest_alphas[i]}')
plt.legend()
plt.xscale('log')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.title('Loss as a function of iterations of 3 smallest alpha')
plt.show()

###########################################################################
#                             END OF YOUR CODE                            #
###########################################################################

This is yet another sanity check. This function plots the regression lines of your model and the model based on the pseudo-inverse calculation. Both models should exhibit the same trend through the data.

In [ ]:
plt.figure(figsize=(7, 7))
plt.plot(X_train[:,1], y_train, 'ro', ms=1, mec='k')
plt.ylabel('Price in USD')
plt.xlabel('sq.ft')
plt.plot(X_train[:, 1], np.dot(X_train, theta), 'o')
plt.plot(X_train[:, 1], np.dot(X_train, theta_pinv), '-')

plt.legend(['Training data', 'Linear regression', 'Best theta']);

## Part 2: Multivariate Linear Regression (30 points)

In most cases, you will deal with databases that have more than one feature. It can be as little as two features and up to thousands of features. In those cases, we use a multiple linear regression model. The regression equation is almost the same as the simple linear regression equation:

$$
\hat{y} = h_\theta(\vec{x}) = \theta^T \vec{x} = \theta_0 + \theta_1 x_1 + ... + \theta_n x_n
$$


If you wrote vectorized code, this part should be straightforward. If your code is not vectorized, you should go back and edit your functions such that they support both multivariate and single variable regression. **Your code should not check the dimensionality of the input before running**.

In [ ]:
# Read comma separated data
df = pd.read_csv('data.csv')
df.head()

## Preprocessing

Like in the single variable case, we need to create a numpy array from the dataframe. Before doing so, we should notice that
some features are clearly irrelevant.

In [ ]:
X = df.drop(columns=['price', 'id', 'date']).values
y = df['price'].values

Use the **same** `preprocess` function you implemented previously. Notice that proper vectorized implementation should work regardless of the dimensionality of the input. You might want to check that your code in the previous parts still works.

In [ ]:
# preprocessing
X, y = preprocess(X, y)

In [ ]:
# training and validation split
np.random.seed(42)
indices = np.random.permutation(X.shape[0])
idx_train, idx_val = indices[:int(0.8*X.shape[0])], indices[int(0.8*X.shape[0]):]
X_train, X_val = X[idx_train,:], X[idx_val,:]
y_train, y_val = y[idx_train], y[idx_val]

Using 3D visualization, we can still observe trends in the data. Visualizing additional dimensions requires advanced techniques we will learn later in the course.

In [ ]:
%matplotlib inline
import mpl_toolkits.mplot3d.axes3d as p3
fig = plt.figure(figsize=(5,5))
ax = p3.Axes3D(fig)
xx = X_train[:, 1][:1000]
yy = X_train[:, 2][:1000]
zz = y_train[:1000]
ax.scatter(xx, yy, zz, marker='o')
ax.set_xlabel('bathrooms')
ax.set_ylabel('sqft_living')
ax.set_zlabel('price')
plt.show()

Use the bias trick again (add a column of ones as the zeroth column in the both the training and validation datasets).

In [ ]:
###########################################################################
#                            START OF YOUR CODE                           #
###########################################################################
X_train =  np.column_stack((np.ones(X_train.shape[0]), X_train))
X_val =  np.column_stack((np.ones(X_val.shape[0]), X_val))
###########################################################################
#                             END OF YOUR CODE                            #
###########################################################################

Make sure the functions `compute_cost` (10 points), `gradient_descent` (15 points), and `pinv` (5 points) work on the multi-dimensional dataset. If you make any changes, make sure your code still works on the single variable regression model. 

In [ ]:
shape = X_train.shape[1]
theta = np.ones(shape)
J = compute_cost(X_train, y_train, theta)

In [ ]:
np.random.seed(42)
shape = X_train.shape[1]
theta = np.random.random(shape)
iterations = 40000
theta, J_history = gradient_descent(X_train ,y_train, theta, best_alpha, iterations)
# theta, J_history = gradient_descent(X_train ,y_train, theta, 1, iterations)

In [ ]:
theta_pinv = pinv(X_train ,y_train)
J_pinv = compute_cost(X_train, y_train, theta_pinv)

We can use visualization to make sure the code works well. Notice we use logarithmic scale for the number of iterations, since gradient descent converges after ~500 iterations.

In [ ]:
plt.plot(np.arange(iterations), J_history)
plt.xscale('log')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.title('Loss as a function of iterations - multivariate linear regression')
plt.hlines(y = J_pinv, xmin = 0, xmax = len(J_history), color='r',
           linewidth = 1, linestyle = 'dashed')
plt.show()

## Part 3: Polynomial Regression (10 points)

Linear Regression allows us to explore linear relationships but if we need a model that describes non-linear dependencies we can also use Polynomial Regression. In order to perform polynomial regression, we create additional features using a function of the original features and use standard linear regression on the new features. For example, consider the following single variable $(x)$ cubic regression:

$$ x_0 = 1,    x_1 = x,    x_2 = x^2,    x_3 = x^3$$

And after using standard linear regression:

$$ f(x) = \theta_0 + \theta_1 x + \theta_2 x^2 +  \theta_3 x^3$$

As required. 

For this exercise, use polynomial regression by using all **quadratic** feature combinations: 

$$ 1, x, y, z, x^2, y^2, z^2, xy, xz, yz, ...$$

and evaluate the MSE cost on the training and testing datasets.

In [ ]:
columns_to_drop = ['price', 'id', 'date']
all_features = df.drop(columns=columns_to_drop)
all_features.head(5)


In order to create polynomial model we need to enlarge the data and the coefficients of theta.
After handling the data by **adding** quadratic combinations **as features** we can use the same algorithm of linear regression to find our best theta

In [ ]:
### Your code here ###
# adding all quadratic combinations as features
X_quad = all_features.values
y_quad = df['price'].values
X_len = X_quad.shape[1]


In [ ]:
# append all quadratic feature combinations as new features
for i in range(X_len):
    for j in range(i,X_len):
        res = X[:, i] * X[:,j]
        X_quad = np.column_stack((X_quad, res))

X_quad, y_quad = preprocess(X_quad, y_quad)  #should be after multi-features
# Bias to create
X_quad.shape
X_quad = np.column_stack((np.ones(X_quad.shape[0]),X_quad))




In [ ]:
# training and validation quadratic set split:
#indices calculated above on Linear multi-features

idx_train_q, idx_val_q = indices[:int(0.8*X_quad.shape[0])], indices[int(0.8*X_quad.shape[0]):]
X_quad_train, X_quad_val = X_quad[idx_train_q], X_quad[idx_val_q]
y_quad_train, y_quad_val = y_quad[idx_train_q], y_quad[idx_val_q]


In [ ]:
# training model
shape_quad = X_quad_train.shape[1]
theta_quad = np.ones(shape_quad)
iterations = 40_000


theta_quad_trained, J_quad_history = efficient_gradient_descent(X_quad_train, y_quad_train, theta_quad, best_alpha, iterations)


In [ ]:
#based on previous training above (multi features)
mse_lin_train = compute_cost(X_train, y_train, theta)
mse_lin_val = compute_cost(X_val, y_val, theta)

#calculate polynomial MSE on training set and validation set.
mse_pol_train = compute_cost(X_quad_train, y_quad_train, theta_quad_trained)
mse_pol_val = compute_cost(X_quad_val, y_quad_val, theta_quad_trained)

lowest_model_train = "Linear model" if mse_lin_train < mse_pol_train else "Polynomial model"
lowest_model_val = "Linear model" if mse_lin_val < mse_pol_val else "Polynomial model"

print('******MSE on training set*******')
print(f'Linear reg= {mse_lin_train}, Polynomial reg= {mse_pol_train}')
print("lowest MSE on training set is " + lowest_model_train)
print()
print('******MSE on validation set*******')
print(f'Linear reg= {mse_lin_val}, Polynomial reg= {mse_pol_val}')
print("lowest MSE on validation set is " + lowest_model_val)

Give an explanations to the results and compare them to regular linear regression. Do they make sense?

The polynomial model has lower MSE to our validation set, and is closest to the results.
the output on the cell above show our performance comparing polynomial and linear model on the same input.
 We can assume that the polynomial is over-fitting our dataset and the fact that our linear model has larger MSE
  on the training set, it does not say it's not optimized for our wanted result.
  More over we can obtain the difference between them is not as big but might be a difference in some fields.


## Part 4: Adaptive Learning Rate (10 points)

So far, we kept the learning rate alpha constant during training. However, changing alpha during training might improve convergence in terms of the global minimum found and running time. Implement the adaptive learning rate method based on the gradient descent algorithm above. 

**Your task is to find proper hyperparameter values for the adaptive technique and compare this technique to the constant learning rate. Use clear visualizations of the validation loss and the learning rate as a function of the iteration**.

Time based decay: this method reduces the learning rate every iteration according to the following formula:

$$\alpha = \frac{\alpha_0}{1 + D \cdot t}$$

Where $\alpha_0$ is the original learning rate, $D$ is a decay factor and $t$ is the current iteration.


In [ ]:
# efficient_gradient_descent based multivariate learning
def eff_adaptive_learning_rate(X, y, theta, alpha_start, decay, iterations):
    #this function to find the best decay

    J_history = [] # Use a python list to save cost in every iteration
    theta = theta.copy() # avoid changing the original thetas
    MIN_DIFF = 1e-8
    for i in range(iterations):
        #same code from gradient_descent() above
        gradient = X.dot(theta) - y
        alpha = alpha_start / (1+ decay * i)
        # new alpha is changing iff decay is givven to function
        theta = theta - alpha * (1/X.shape[0]) * X.T.dot(gradient)
        J_history.append(compute_cost(X, y, theta))
        if (i > 0) and (J_history[-2] - J_history[-1] < MIN_DIFF):
            break
    return theta, J_history


def find_best_decay(X_train, y_train, X_val, y_val, alpha_start, iterations):
    # random numbers
    decays_list = [0.0001, 0.001, 0.01, 0.1, 0.5, 1, 2]
    decay_dict = {}
    np.random.seed(42)
    theta_start = np.random.random(size=X_train.shape[1])
    for decay in decays_list:
        theta_cur, _ = eff_adaptive_learning_rate(X_train, y_train, theta_start, alpha_start, decay, iterations)
        decay_dict[decay] = compute_cost(X_val, y_val, theta_cur)
    return decay_dict

In [ ]:
iterations = 10_000
alpha_start = 2
decay_dict={}

# Data Preprocessing
df = pd.read_csv('data.csv')
X = df.drop(columns=['price', 'id', 'date']).values
y = df['price'].values

# Preprocessing
X, y = preprocess(X, y)

# training and validation split
np.random.seed(42)
indices = np.random.permutation(X.shape[0])
idx_train, idx_val = indices[:int(0.8*X.shape[0])], indices[int(0.8*X.shape[0]):]
X_train, X_val = X[idx_train], X[idx_val]
y_train, y_val = y[idx_train], y[idx_val]

# Bias Trick
X_train =  np.column_stack((np.ones(X_train.shape[0]), X_train))
X_val =  np.column_stack((np.ones(X_val.shape[0]), X_val))

#find best decay
decay_dict = find_best_decay(X_train, y_train, X_val, y_val, alpha_start, iterations)
# print(decay_dict)

In [ ]:
best_decay = min(decay_dict, key=decay_dict.get)
print(f'The best decay found on alpha={alpha_start} is :{best_decay}')
###
sorted_decay = sorted(decay_dict, key=decay_dict.get)
# We show the best the avg and worst decays
three_decays = [sorted_decay[0], sorted_decay[len(sorted_decay) // 2], sorted_decay[-1]]
theta_arr = []
J_history_array = []
# start from the same theta by seeded random
np.random.seed(42)
theta_1 = np.random.random(size=X_train.shape[1])
iterations = 10_000
# training for each decay
#calculate no decay - start on decay 0 - constant
_, J_eff = eff_adaptive_learning_rate(X_train ,y_train, theta_1, alpha_start, 0, iterations)
J_history_array.append(J_eff)
#for non zero decays - we check only best ones
for decay in sorted_decay[0:4]:
    theta_temp, J_eff = eff_adaptive_learning_rate(X_train ,y_train, theta_1, alpha_start, decay, iterations)
    J_history_array.append(J_eff)
    theta_arr.append(theta_temp)

In [ ]:
plt.plot(np.arange(len(J_history_array[0])), J_history_array[0], label=f'constant (decay =0)')
for i in range(1,len(J_history_array)): # [1-3]
    plt.plot(np.arange(len(J_history_array[i])), J_history_array[i], label=f'decay={sorted_decay[i]}')

plt.legend()
plt.xscale('log')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.title('Loss as a function of iterations of smallest decays')
plt.show()


We can check on the different decays on arbitrary alpha, we can get even better results if on each decay we will
find the best alpha, but we did not perform, it took too much running time (seems irrelevant for this use).
all the model eventually
